# Coletando daos IBGE, Armazenando no BigQuery e Preenchendo PPT

**Vamos divir o processo em duas etapas para facilidar a Didatica**
- Primeira Etapa: Coleta de dados do IBGE e Armazenamento no BigQuery
- Segunda Etapa: Consumo dos dados do BigQuery e Prenchimento do PPT


### Primeira Etapa: Coleta de dados do IBGE e Armazenamento no BigQuery
Neste Etapa vamos coletar dados do IBGE utilizando o SIDRA (Sistema IBGE de Recuperação Automática) que é uma plataforma do IBGE criada com o objetivo de consultar as pesquisas criados e disponibilizadas pelo Instituto. A interface permite que sejam acessado os dados através de API's. 

O objetivo desta etapa será mostrar como é possível acessar estes dados utilizando o Python, para exemplificar estarei utilizando a tabela de estimativa populacional por Municipio.

Existe um módulo python chamado sidrapy que permite baixar facilmente estes dados e realizar a importação direta para um dataframe Pandas.

Referências que utilizei para construir o Artigo:
- https://pypi.org/project/sidrapy/ => Documentação oficial módulo sidrapy no pipy
- https://sidrapy.readthedocs.io/pt_BR/latest/modules/table.html => documentação do sidrapy (achei esta mais detalhada)
- https://sidra.ibge.gov.br/acervo => Pagina para consultar tableas disponiveis
- https://apisidra.ibge.gov.br/ => Documentação da API

> Primeiramente precisaremos entrar no pompt de comando e instalar a bibliotecas necessarias através do arquivo requirements.txt atrvés dp comando 
> 
> ``` pip install -r requirements.txt ```
> 




**Importa Libs**

In [12]:
import sidrapy
from google.cloud import bigquery
from google.oauth2 import service_account

**Baixando a Tabela de Estimativa de Populção residente por Municipio**

Estaremos Utilizando os ultimos 6 anos da Tabela 6579 - População residente estimada agregada por Municipio

Descrição da Serie a ser utilizada: https://apisidra.ibge.gov.br/desctabapi.aspx?c=6579

Descrição dos parametros que utilizaremos:
- **table_code** = Código da tabela de onde se deseja extrair os dados, utilizaremos a *6579 - População Residente Estimada*
- **territorial_level** = Nível territorial do IBGE , utilizaremos a *6 - Município*
- **ibge_territorial_code** = Unidades territoriais do IBGE. É possível especificar múltiplos elementos utilizando vírgulas e utilizaremos toddos os disponiveis, estaremos utilizando o parametro "all" informando que queremos todos
- **period** = Períodos (meses, anos etc.) desejados. Caso não seja especificado, traz apenas os períodos mais recentes, utilizaremos os ultimos 6 anos
- **header** = Especifica se o resultado será precedido por um registro de cabeçalho. Caso não seja espefificado, traz o cabeçalho.
    - Opções:
        - ‘y’ - Traz o cabeçalho
        - ‘n’ - Não traz o cabeçalho'n'
    
    não iremos trazer o cabeçalho, pois vamos ajustar

In [5]:
dfPopulacaoResidente = sidrapy.get_table(table_code="6579", territorial_level="6", ibge_territorial_code="all", period="last 1", header='n')

In [6]:
dfPopulacaoResidente

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,6,Município,45,Pessoas,22516,1100015,Alta Floresta D'Oeste - RO,2021,2021,9324,População residente estimada
1,6,Município,45,Pessoas,111148,1100023,Ariquemes - RO,2021,2021,9324,População residente estimada
2,6,Município,45,Pessoas,5067,1100031,Cabixi - RO,2021,2021,9324,População residente estimada
3,6,Município,45,Pessoas,86416,1100049,Cacoal - RO,2021,2021,9324,População residente estimada
4,6,Município,45,Pessoas,16088,1100056,Cerejeiras - RO,2021,2021,9324,População residente estimada
...,...,...,...,...,...,...,...,...,...,...,...
5565,6,Município,45,Pessoas,14088,5222005,Vianópolis - GO,2021,2021,9324,População residente estimada
5566,6,Município,45,Pessoas,9002,5222054,Vicentinópolis - GO,2021,2021,9324,População residente estimada
5567,6,Município,45,Pessoas,6451,5222203,Vila Boa - GO,2021,2021,9324,População residente estimada
5568,6,Município,45,Pessoas,5941,5222302,Vila Propício - GO,2021,2021,9324,População residente estimada


**Ajustando data frame**

Quando baixamos o dataframe ele vem com os campos abaixo: 
- **NC**  = Nível Territorial (Código)
- **MN**  = Nível Territorial
- **MC**  = Unidade de Medida (Código)
- **MN**  = Unidade de Medida
- **V**   = Valor
- **D1C** = Unidade da Federação (Código)
- **D1N** = Unidade da Federação
- **D2C** = Ano (Código)
- **D2N** = Ano
- **D3C** = Variável (Código)	
- **D3N** = Variável	

Porém para nossa analise precisaremos apenas de alguns campos que são:
- **MN**  = Nível Territorial
- **V**   = Valor
- **D1N** = Unidade da Federação - Este campo sera dividido em Municipio e UF
- **D2N** = Ano
- **D3N** = Variável	

Então vamos tratar os dados e deixar apenas os campos necessarios e aproveitar para renomear as colunas


In [7]:
newColumns = dfPopulacaoResidente["D1N"].str.split(" - ", n = 1, expand = True)
dfPopulacaoResidente['municipio']= newColumns[0]
dfPopulacaoResidente['uf']= newColumns[1]
dfPopulacaoResidente.pop("D1N")
dfPopulacaoResidente.pop("NC")
dfPopulacaoResidente.pop("MC")
dfPopulacaoResidente.pop("D1C")
dfPopulacaoResidente.pop("D2C")
dfPopulacaoResidente.pop("D3C")
dfPopulacaoResidente.pop("NN")

dfPopulacaoResidente

,MN,V,D2N,D3N,municipio,uf
0,Pessoas,22516,2021,População residente estimada,Alta Floresta D'Oeste,RO
1,Pessoas,111148,2021,População residente estimada,Ariquemes,RO
2,Pessoas,5067,2021,População residente estimada,Cabixi,RO
3,Pessoas,86416,2021,População residente estimada,Cacoal,RO
4,Pessoas,16088,2021,População residente estimada,Cerejeiras,RO
...,...,...,...,...,...,...
5565,Pessoas,14088,2021,População residente estimada,Vianópolis,GO
5566,Pessoas,9002,2021,População residente estimada,Vicentinópolis,GO
5567,Pessoas,6451,2021,População residente estimada,Vila Boa,GO
5568,Pessoas,5941,2021,População residente estimada,Vila Propício,GO


### Ajustando o nome das Colunas

In [8]:
dfPopulacaoResidente.columns = ['unidadeMedida',	'valor', 'ano', 'totalizador', 'municipio', 'uf']
dfPopulacaoResidente = dfPopulacaoResidente.astype({
    'valor': 'int64'
})

dfPopulacaoResidente

,unidadeMedida,valor,ano,totalizador,municipio,uf
0,Pessoas,22516,2021,População residente estimada,Alta Floresta D'Oeste,RO
1,Pessoas,111148,2021,População residente estimada,Ariquemes,RO
2,Pessoas,5067,2021,População residente estimada,Cabixi,RO
3,Pessoas,86416,2021,População residente estimada,Cacoal,RO
4,Pessoas,16088,2021,População residente estimada,Cerejeiras,RO
...,...,...,...,...,...,...
5565,Pessoas,14088,2021,População residente estimada,Vianópolis,GO
5566,Pessoas,9002,2021,População residente estimada,Vicentinópolis,GO
5567,Pessoas,6451,2021,População residente estimada,Vila Boa,GO
5568,Pessoas,5941,2021,População residente estimada,Vila Propício,GO


### Criando um Dataframe Novo com os dados Agrupados por UF

In [9]:
dfPopulacaoGroupUF = dfPopulacaoResidente.groupby(["unidadeMedida", 'ano', 'totalizador', 'uf']).agg({'valor': ['sum']}).reset_index()
dfPopulacaoGroupUF

,unidadeMedida,ano,totalizador,uf,valor
,,,,,sum
0,Pessoas,2021,População residente estimada,AC,906876
1,Pessoas,2021,População residente estimada,AL,3365351
2,Pessoas,2021,População residente estimada,AM,4269995
3,Pessoas,2021,População residente estimada,AP,877613
4,Pessoas,2021,População residente estimada,BA,14985284
5,Pessoas,2021,População residente estimada,CE,9240580
6,Pessoas,2021,População residente estimada,DF,3094325
7,Pessoas,2021,População residente estimada,ES,4108508
8,Pessoas,2021,População residente estimada,GO,7206589


### Ajustando nome das colunas do DataFrame agrupado

In [10]:
dfPopulacaoGroupUF.columns = ['unidadeMedida', 'ano', 'totalizador', 'uf', 'valor']
dfPopulacaoGroupUF

,unidadeMedida,ano,totalizador,uf,valor
0,Pessoas,2021,População residente estimada,AC,906876
1,Pessoas,2021,População residente estimada,AL,3365351
2,Pessoas,2021,População residente estimada,AM,4269995
3,Pessoas,2021,População residente estimada,AP,877613
4,Pessoas,2021,População residente estimada,BA,14985284
5,Pessoas,2021,População residente estimada,CE,9240580
6,Pessoas,2021,População residente estimada,DF,3094325
7,Pessoas,2021,População residente estimada,ES,4108508
8,Pessoas,2021,População residente estimada,GO,7206589
9,Pessoas,2021,População residente estimada,MA,7153262


### Autenticando no Google Cloud
**Neste ponto iremos utilizar a chave de serviço do GCP**

**Obs:** Caso não saiba gerar a chave pode utilizar o link a seguir:
- https://medium.com/@alexandretavares.bigdata/criando-uma-conta-de-servi%C3%A7o-no-gcp-eeac59f3cbcb


In [11]:
# Informe aqui o caminho onde a chave estará armazenada
# Recomendamos que guarde esta chave em um cofre digital para maior segurança
pathCredencial = "key/artmedium.json" 

credentials = service_account.Credentials.from_service_account_file(
    pathCredencial,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

### Gravando os Dados no BigQuery

In [ ]:
tbPessoasUF = 'raw.tb_nat_populacao_uf'
tbPessoasMunicipio = 'raw.tb_nat_populacao_municipio'

job_config = bigquery.LoadJobConfig(
    autodetect = True, 
    write_disposition = 'WRITE_TRUNCATE'
    )

job = client.load_table_from_dataframe(
    dfPopulacaoGroupUF, 
    tbPessoasUF, 
    job_config=job_config,
)

job2 = client.load_table_from_dataframe(
    dfPopulacaoResidente, 
    tbPessoasMunicipio, 
    job_config=job_config,
)